In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import display

from datetime import date
import datetime

In [2]:
locations = {'Slovenska': '9000000619',
             'Dunajska1': '9000001783',
             'Dunajska2': '9000000656',
             'Dunajska3': '9000000655',
             'Izanska1': '9000000820',
             'Izanska2': '9000001506',
             'Celovska': '9000001950',
             'Gosposvetska': '9000000529',
             'Koper': '9000001092'}


## Settings

In [3]:
separate_directions = True
remove_outliers = True

pct_up_threshold = 0.5

## Read the counters

In [4]:
df = pd.DataFrame()

for label, loc_id in locations.items():
    file_name = 'data\\'+label+'_telraam.csv'
    df2 = pd.read_csv(file_name)
    df2['location'] = label
    df2['counter_id'] = loc_id
    print(label+"; from:", df2['date'].min()+"; to:", df2['date'].max())
    
    df = df.append(df2, ignore_index=True)

df = df[df['pct_up'] != 0]
if pct_up_threshold:
    df = df[df['pct_up'] >= pct_up_threshold]

df.reset_index(drop=True, inplace=True)

Slovenska; from: 2020-08-25T11:00:00.000Z; to: 2021-05-07T10:00:00.000Z
Dunajska1; from: 2021-03-16T14:00:00.000Z; to: 2021-05-07T10:00:00.000Z
Dunajska2; from: 2020-09-03T10:00:00.000Z; to: 2021-05-24T07:00:00.000Z
Dunajska3; from: 2020-09-03T08:00:00.000Z; to: 2021-05-25T06:00:00.000Z
Izanska1; from: 2020-10-06T19:00:00.000Z; to: 2021-05-18T08:00:00.000Z
Izanska2; from: 2021-01-28T17:00:00.000Z; to: 2021-05-18T08:00:00.000Z
Celovska; from: 2021-04-19T10:00:00.000Z; to: 2021-05-18T08:00:00.000Z
Gosposvetska; from: 2020-07-26T06:00:00.000Z; to: 2021-02-23T19:00:00.000Z
Koper; from: 2020-12-03T08:00:00.000Z; to: 2021-05-18T08:00:00.000Z


In [5]:
#df['pedestrian']*df['pct_up']

## Dates
* Convert from UTC to local time
* get dates and times
* get workday data

In [6]:
df['utc'] = df['date'].map(lambda d:datetime.datetime.strptime(d,'%Y-%m-%dT%H:%M:%S.%fZ'))
df['local'] = df['utc'].map(lambda x:datetime.datetime.fromtimestamp(x.timestamp()))

In [7]:
#df['day'] = df['local'].map(date.weekday)
#df['hour'] = pd.DatetimeIndex(df['local']).hour
df['date'] = pd.DatetimeIndex(df['local']).date
df['time'] = pd.DatetimeIndex(df['local']).hour + pd.DatetimeIndex(df['local']).minute/60 + pd.DatetimeIndex(df['local']).second/3600

In [8]:
df['weekday'] = df['local'].dt.weekday
df['workday'] = df['weekday'].map(lambda x: 1 if x < 5 else 0)
df = df.drop(columns = "weekday")
df['workday'].describe()

count    13588.000000
mean         0.723285
std          0.447391
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: workday, dtype: float64

## Weather data

In [9]:
df_weather = pd.read_csv('data\\weather.csv')
#df_weather['date'] = pd.to_datetime(df_weather['date'])
for date in df['date'].unique():
    try:
        weather = df_weather.loc[df_weather['date'] == str(date), 'weather'].iloc[0]
    except:
        weather = np.nan
    locs = date == df['date']
    df.loc[locs, "weather"] = weather

## Editing the columns

In [10]:
df = df.drop(columns=['utc', 'local'])

In [11]:
df = df[['counter_id', 'date', 'time', 'pedestrian', 'bike', 'car', 'lorry', 'pedestrian_lft',
       'bike_lft', 'car_lft', 'lorry_lft', 'pedestrian_rgt', 'bike_rgt',
       'car_rgt', 'lorry_rgt', 'location', 'pct_up', 'workday', 'weather']]

## Saving

In [12]:
df.to_csv("data\\telraam_full.csv", index=False)

In [13]:
df

,counter_id,date,time,pedestrian,bike,car,lorry,pedestrian_lft,bike_lft,car_lft,lorry_lft,pedestrian_rgt,bike_rgt,car_rgt,lorry_rgt,location,pct_up,workday,weather
0,9000000619,2020-08-25,15.0,0.000000,106.072584,373.841179,0.000000,0.000000,27.164930,243.190801,0.000000,0.000000,78.907654,130.650377,0.000000,Slovenska,0.773056,1,good
1,9000000619,2020-08-25,16.0,17.307692,93.195266,420.710059,0.000000,6.656805,14.644970,244.970414,0.000000,10.650888,78.550296,175.739645,0.000000,Slovenska,0.751111,1,good
2,9000000619,2020-08-25,17.0,14.137808,102.820421,341.877901,0.000000,1.285255,8.996787,167.083185,0.000000,12.852553,93.823634,174.794716,0.000000,Slovenska,0.778056,1,good
3,9000000619,2020-08-25,18.0,13.928948,74.709814,307.703130,0.000000,5.065072,8.863876,150.685895,0.000000,8.863876,65.845937,157.017235,0.000000,Slovenska,0.789722,1,good
4,9000000619,2020-08-25,19.0,9.870740,73.325499,332.784959,0.000000,4.230317,9.870740,146.650999,0.000000,5.640423,63.454759,186.133960,0.000000,Slovenska,0.709167,1,good
5,9000000619,2020-08-26,6.0,24.896266,49.792531,258.921162,0.000000,6.639004,9.958506,101.244813,0.000000,18.257261,39.834025,157.676349,0.000000,Slovenska,0.602500,1,good
6,9000000619,2020-08-26,7.0,30.000000,178.500000,222.000000,0.000000,1.500000,33.000000,84.000000,0.000000,28.500000,145.500000,138.000000,0.000000,Slovenska,0.666667,1,good
7,9000000619,2020-08-26,8.0,24.040067,106.677796,339.565943,0.000000,4.507513,18.030050,142.737896,0.000000,19.532554,88.647746,196.828047,0.000000,Slovenska,0.665556,1,good
8,9000000619,2020-08-26,9.0,8.845209,114.987715,219.361179,0.000000,3.538084,24.766585,107.911548,0.000000,5.307125,90.221130,111.449631,0.000000,Slovenska,0.565278,1,good
9,9000000619,2020-08-26,10.0,3.142732,98.996072,257.704059,0.000000,1.571366,31.427324,138.280227,0.000000,1.571366,67.568747,119.423832,0.000000,Slovenska,0.636389,1,good


### Smaller dataset

In [14]:
df['count'] = df['car'] + df['lorry']
df['count_lft'] = df['car_lft'] + df['lorry_lft']
df['count_rgt'] = df['car_rgt'] + df['lorry_rgt']

In [15]:
df_small = pd.DataFrame(columns = ['counter_id','date','time','count','workday','weather'])
counter_ids = df.counter_id.unique()

for counter_id in counter_ids:
    #df1 = df[df['counter_id'] == counter_id]
    df1 = df[df['counter_id'].str.startswith(counter_id)]

    
    if separate_directions:
        df11 = df1[['counter_id','date','time','count_lft','workday','weather']].copy()
        df11['counter_id'] = df11['counter_id'] + '-1'
        df12 = df1[['counter_id','date','time','count_rgt','workday','weather']].copy()
        df12['counter_id'] = df12['counter_id'] + '-2'
    
        df11.columns = ['counter_id','date','time','count','workday','weather']
        df12.columns = ['counter_id','date','time','count','workday','weather']

        df_small= pd.concat((df_small, df11), ignore_index=True)
        df_small= pd.concat((df_small, df12), ignore_index=True)
    else:
        df1 = df1[['counter_id','date','time','count','workday','weather']]
        df_small= pd.concat((df_small, df1), ignore_index=True)
        

In [16]:
df_small.shape

(27176, 6)

### Remove outliers

In [17]:
if remove_outliers:
    df_small = df_small[df_small['count'] != 0]        
    df = pd.DataFrame()
    for counter_id in counter_ids:
        #df2 = df_small[df_small['counter_id'] == counter_id]   
        df2 = df_small[df_small['counter_id'].str.startswith(counter_id)]

        m, s = np.mean(df2['count']), np.std(df2['count'])
    
        idxs = np.logical_not(np.logical_or(df2['count'] > m + 3 * s, df2['count'] < m - 3 * s))
        df2 = df2[idxs]
        
        df = pd.concat([df,df2], ignore_index=True, sort=False)
        
        
else:
    df = df_small
    

In [18]:
df.shape

(25452, 6)

## Saving

In [19]:
df.to_csv("data\\telraam.csv", index=False)

In [20]:
df

,counter_id,date,time,count,workday,weather
0,9000000619-1,2020-08-25,15.0,243.190801,1,good
1,9000000619-1,2020-08-25,16.0,244.970414,1,good
2,9000000619-1,2020-08-25,17.0,167.083185,1,good
3,9000000619-1,2020-08-25,18.0,150.685895,1,good
4,9000000619-1,2020-08-25,19.0,146.650999,1,good
5,9000000619-1,2020-08-26,6.0,101.244813,1,good
6,9000000619-1,2020-08-26,7.0,84.000000,1,good
7,9000000619-1,2020-08-26,8.0,142.737896,1,good
8,9000000619-1,2020-08-26,9.0,107.911548,1,good
9,9000000619-1,2020-08-26,10.0,138.280227,1,good
